In [1]:
import feedparser

def fetchNewsFromTheArticles(rss_feeds):
    articles = []
    try:
        for url in rss_feeds:
            feed = feedparser.parse(url)
            print(feed)
            for entry in feed.entries:  # use entries instead of entries
                article = {
                    "title": entry.title,
                    "link": entry.link,
                    "summary": entry.summary,
                    "published": entry.published if 'published' in entry else "no date found"
                }
                articles.append(article)

    except Exception as e:
        print(f"Error fetching news from {url}: {e}")

    return articles

# Example usage
rss_feeds = [
    "http://feeds.bbci.co.uk/news/world/rss.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/HomePage.xml",
    "https://feeds.npr.org/1001/rss.xml",
    "https://www.aljazeera.com/xml/rss/all.xml"
]

articles = fetchNewsFromTheArticles(rss_feeds)
print(articles[0:2])


{'bozo': False, 'entries': [{'title': "'Like a tsunami': Spaniards recount horror of deadly floods", 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://feeds.bbci.co.uk/news/world/rss.xml', 'value': "'Like a tsunami': Spaniards recount horror of deadly floods"}, 'summary': 'Flash floods sparked by torrential rainfall have killed 95 people around Valencia, while dozens are still missing.', 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://feeds.bbci.co.uk/news/world/rss.xml', 'value': 'Flash floods sparked by torrential rainfall have killed 95 people around Valencia, while dozens are still missing.'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.bbc.com/news/articles/c8ekdp4r6kpo'}], 'link': 'https://www.bbc.com/news/articles/c8ekdp4r6kpo', 'id': 'https://www.bbc.com/news/articles/c8ekdp4r6kpo#0', 'guidislink': False, 'published': 'Wed, 30 Oct 2024 18:37:14 GMT', 'published_parsed': time.struct_time(tm_year=2024

In [2]:
from transformers import pipeline

# Load a pre-trained sentiment analysis model
sentiment_classifier = pipeline("sentiment-analysis")

def filter_good_news(articles):
    good_news = []
    for article in articles:
        sentiment = sentiment_classifier(article['title'])[0]
        if sentiment['label'] == 'POSITIVE' and sentiment['score'] > 0.9:
            good_news.append(article)
    return good_news

# Filter only positive news articles
good_news_articles = filter_good_news(articles)
print("the new are good news below the one")
print(good_news_articles[:5])
print(f"Good News Articles: {len(good_news_articles)} found.")

C:\Users\gopin\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


the new are good news below the one
[{'title': 'Ukraine in new mobilisation drive as Russia advances', 'link': 'https://www.bbc.com/news/articles/cx2n4n1r307o', 'summary': "Ukraine's military has been under severe pressure of late, in part due to Russia's greater manpower.", 'published': 'Wed, 30 Oct 2024 06:06:16 GMT'}, {'title': 'China declares success as its youngest astronauts reach space', 'link': 'https://www.bbc.com/news/articles/c5yp47prg7jo', 'summary': 'The BBC’s Laura Bicker sees the Shenzhou 19 spacecraft take off in a record year of space exploration.', 'published': 'Wed, 30 Oct 2024 06:16:40 GMT'}, {'title': 'Lisa Kudrow pays tribute to Friends mother Teri Garr', 'link': 'https://www.bbc.com/news/articles/cje09595v5ko', 'summary': 'Kudrow says the Oscar-nominated actress, who played her on-screen mum, was "a comedic acting genius".', 'published': 'Wed, 30 Oct 2024 11:28:28 GMT'}, {'title': 'China spacecraft launches in mission to space station', 'link': 'https://www.bbc.c

In [3]:
def create_newsletter(good_news_articles):
    newsletter_content = "Good News Newsletter\n\n"
    
    for article in good_news_articles:
        newsletter_content += f"**Title:** {article['title']}\n"
        newsletter_content += f"**Link:** {article['link']}\n"
        newsletter_content += f"**Summary:** {article['summary']}\n"
        newsletter_content += f"**Published:** {article['published']}\n"
        newsletter_content += "\n---\n\n"  # Separator
    
    return newsletter_content

In [7]:
import requests

def send_newsletter_to_model(newsletter):
    url = "http://127.0.0.1:1234/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    
    payload = {
        "messages": [
            {
                "role": "user",
                "content": "Format this newsletter:\n\n" + newsletter
            }
        ],
        "model": "llama-3b"   
    }
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        summary = response.json()["choices"][0]["message"]["content"]
        print(summary)
        return summary
    else:
        print("Error:", response.status_code, response.text)


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas


def save_summary_to_pdf(summary):
    pdf_filename = "formatted_newsletter.pdf"
    
    
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    width, height = letter
    
     
    c.setFont("Helvetica-Bold", 16)
    c.drawString(72, height - 72, "Formatted Newsletter")
    
     
    c.setFont("Helvetica", 8)
    text_object = c.beginText(72, height - 100)
    text_object.setTextOrigin(72, height - 100)
    text_object.setLeading(14)   

    
    for line in summary.split('\n'):
        text_object.textLine(line)
    
    c.drawText(text_object)
    c.save()   
    print(f"Newsletter saved as {pdf_filename}")

In [8]:
newsletter=create_newsletter(good_news_articles)
newletterSummary=send_newsletter_to_model(newsletter)
save_summary_to_pdf(newletterSummary)

Here is the formatted newsletter:


**Good News Newsletter**

**Title:** Ukraine in new mobilisation drive as Russia advances
**Link:** https://www.bbc.com/news/articles/cx2n4n1r307o
**Summary:** Ukraine's military has been under severe pressure of late, in part due to Russia's greater manpower.
**Published:** Wed, 30 Oct 2024 06:06:16 GMT


**Title:** China declares success as its youngest astronauts reach space
**Link:** https://www.bbc.com/news/articles/c5yp47prg7jo
**Summary:** The BBC’s Laura Bicker sees the Shenzhou 19 spacecraft take off in a record year of space exploration.
**Published:** Wed, 30 Oct 2024 06:16:40 GMT


**Title:** Lisa Kudrow pays tribute to Friends mother Teri Garr
**Link:** https://www.bbc.com/news/articles/cje09595v5ko
**Summary:** Kudrow says the Oscar-nominated actress, who played her on-screen mum, was "a comedic acting genius".
**Published:** Wed, 30 Oct 2024 11:28:28 GMT


**Title:** China spacecraft launches in mission to space station
**Link:** https